In [1]:
from __future__ import print_function
# Doing by file until I can figure out OAuth
import os
token_file = os.path.join(os.path.expanduser("~/"), ".jira_auth")
with open(token_file, 'r') as fd:
    uname = fd.readline().strip()  # Can't hurt to be paranoid
    pwd = fd.readline().strip()

SERVER = "https://jira.lsstcorp.org/"
from jira import JIRA
js = JIRA(server=SERVER, basic_auth=(uname, pwd))

In [3]:
query = "project = Simulations AND issuetype = Epic AND summary ~ \"SOCS Release\" ORDER BY key"
issues = js.search_issues(query)
print(len(issues))

12


In [42]:
def get_linked_epics(issue, linkTypeName="Containment"):
    linked_epics = []
    links = issue.fields.issuelinks
    for link in links:
        if link.type.name == linkTypeName:
            linked_epics.append(js.issue(link.outwardIssue.key))
    return linked_epics

In [43]:
def create_list_from_numbered_description(desrc, bulletType="*"):
    import re
    olist = []
    for line in desrc.strip().split(os.linesep):
        values = line.split()
        if re.match("\d+\.\d+", values[0]) is None:
            numbered = bulletType
        else:
            numbered = bulletType * 2
        olist.append("{0} {1}".format(numbered, " ".join(values[1:])))
    return olist

In [44]:
SHOW_ISSUES = True
# Write some boiler plate for the page
page_content = []
page_content.append("This page details the SOCS workplan.")
import time
page_content.append("Updated: {0}".format(time.strftime("%Y-%m-%d %H:%M", time.localtime())))
page_content.append("")

for issue in issues:
    socs_summary = issue.fields.summary
    page_content.append("h2. {0}".format(socs_summary))
    socs_duedate = issue.fields.duedate
    page_content.append("*Release Date:* {0}".format(socs_duedate))
    page_content.append("")
    
    page_content.append("h3. Statement of Work")
    socs_descr = issue.fields.description
    socs_work = create_list_from_numbered_description(socs_descr)
    page_content.append(os.linesep.join(socs_work))
    page_content.append("")
    
    sub_epics = get_linked_epics(issue)
    for sub_epic in sub_epics:
        page_content.append("h4. {0}".format(sub_epic.fields.summary))
        page_content.append("h5. Statement of Work")
        if sub_epic.fields.description is None:
            descr = "No description provided."
        else:
            descr = sub_epic.fields.description
        page_content.append(descr)
        page_content.append("")
        if SHOW_ISSUES:
            page_content.append("h6. Issues")
            se_issues = js.search_issues("'Epic Link' = '{0}'".format(sub_epic.fields.summary))
            page_content.append("Number of Issues = {0}".format(len(se_issues)))
            if len(se_issues) > 0:
                page_content.append("")
                page_content.append("|| Key || Description || Status ||")
                for se_issue in se_issues:
                    page_content.append("| {0} | {1} | {2} |".format(se_issue.key, se_issue.fields.summary,
                                                                    se_issue.fields.status.name))
                
print(os.linesep.join(page_content))    

JIRAError: JiraError HTTP 400
	text: No issues have a parent epic with key or name 'Refactor and integrate environmental model'
	url: https://jira.lsstcorp.org/rest/api/2/search?validateQuery=True&startAt=0&maxResults=50&jql=%27Epic+Link%27+%3D+%27Refactor+and+integrate+environmental+model%27
	response headers = {'x-asessionid': 'g6iu1f', 'date': 'Thu, 24 Sep 2015 19:24:09 GMT', 'x-arequestid': '744x4673031x1', 'transfer-encoding': 'chunked', 'server': 'nginx/1.0.15', 'x-ausername': 'mareuter', 'x-seraph-loginreason': 'OK', 'vary': 'User-Agent', 'cache-control': 'no-cache, no-store, no-transform', 'x-content-type-options': 'nosniff', 'x-asen': 'SEN-2935417', 'content-encoding': 'gzip', 'connection': 'keep-alive', 'content-type': 'application/json;charset=UTF-8'}
	response text = {"errorMessages":["No issues have a parent epic with key or name 'Refactor and integrate environmental model'"],"errors":{}}
